In [19]:
from pandasql import sqldf
import pandas as pd
from datetime import datetime

In [41]:

# 1. Tabela CUSTOMER
customer = pd.DataFrame({
    'costumer_id': ['c001', 'c002', 'c003'],
    'name': ['João Silva', 'Maria Santos', 'Pedro Costa'],
    'document_number': [12345678900, 98765432100, 11122233344],
    'created_at': [datetime.now()] * 3
})

# 2. Tabela ACCOUNT
account = pd.DataFrame({
    'account_id': [1001, 1002, 1003, 1004],
    'costumer_id': ['c001', 'c001', 'c002', 'c003'],
    'status': ['active'] * 4,
    'created_at': [datetime.now()] * 4
})

# 3. Tabela BANKSLIP (boleto)
bankslip = pd.DataFrame({
    'amount': [100.0, 250.0, 150.0, 300.0],
    'account_id': [1001, 1001, 1002, 1003],
    'beneficiary_name': ['Empresa A'] * 4,
    'dt': ['2024-01-15', '2024-01-15', '2024-01-16', '2024-01-15']
})

# 4. Tabela PIX_SEND
pix_send = pd.DataFrame({
    'amount': [50.0, 75.0, 200.0],
    'account_id': [1001, 1002, 1003],
    'dt': ['2024-01-15', '2024-01-15', '2024-01-16']
})

# 5. Tabela PIX_RECEIVED
pix_received = pd.DataFrame({
    'amount': [80.0, 120.0, 90.0],
    'account_id': [1001, 1002, 1003],
    'dt': ['2024-01-15', '2024-01-16', '2024-01-15']
})

# 6. Tabela P2P_TEF
p2p_tef = pd.DataFrame({
    'amount': [150.0, 200.0, 100.0],
    'account_id_source': [1001, 1002, 1003],
    'account_id_destination': [1002, 1003, 1001],
    'dt': ['2024-01-15', '2024-01-15', '2024-01-16']
})

In [95]:
query = """
WITH result_union AS (
    select dt as date, account_id, amount, 'boleto' as transaction_type from bankslip
    union all
    select dt as date, account_id, amount, 'pix' as transaction_type from pix_send
    union all
    select dt as date, account_id, amount, 'pix' as transaction_type from pix_received
    union all
    select dt as date, account_id_destination as account_id, amount, 'p2p' as transaction_type from p2p_tef
    union all
    select dt as date, account_id_source as account_id, amount, 'p2p' as transaction_type from p2p_tef
)
select 
    a.costumer_id, 
    a.account_id, 
    c.name, 
    ru.date, 
    ru.transaction_type, 
    avg(ru.amount) as mean_value 
from customer c 
left join account a on c.costumer_id = a.costumer_id
left join result_union ru on a.account_id = ru.account_id 
where ru.amount is not null
group by a.costumer_id, a.account_id, c.name, ru.date, ru.transaction_type
order by ru.transaction_type, ru.date
"""

result = sqldf(query, locals())
print(result)

   costumer_id  account_id          name        date transaction_type  \
0         c001        1001    João Silva  2024-01-15           boleto   
1         c002        1003  Maria Santos  2024-01-15           boleto   
2         c001        1002    João Silva  2024-01-16           boleto   
3         c001        1001    João Silva  2024-01-15              p2p   
4         c001        1002    João Silva  2024-01-15              p2p   
5         c002        1003  Maria Santos  2024-01-15              p2p   
6         c001        1001    João Silva  2024-01-16              p2p   
7         c002        1003  Maria Santos  2024-01-16              p2p   
8         c001        1001    João Silva  2024-01-15              pix   
9         c001        1002    João Silva  2024-01-15              pix   
10        c002        1003  Maria Santos  2024-01-15              pix   
11        c001        1002    João Silva  2024-01-16              pix   
12        c002        1003  Maria Santos  2024-01-1